# Mendeteksi Anomali pada Jaringan

Anomaly detection adalah teknik pelatihan unsupervised yang menganalisis sejauh mana data yang masuk berbeda dari data yang digunakan untuk melatih jaringan saraf. Secara tradisional, para ahli keamanan siber menggunakan anomaly detection untuk memastikan keamanan jaringan. Namun, dalam ilmu data (data science), teknik ini juga dapat digunakan untuk mendeteksi masukan (input) yang tidak pernah digunakan dalam proses pelatihan jaringan saraf.

Terdapat beberapa dataset yang umum digunakan untuk mendemonstrasikan anomaly detection. Pada kajian kali ini, kita akan membahas dataset KDD-99.
- [Stratosphere IPS Dataset](https://www.stratosphereips.org/category/dataset.html)
- [The ADFA Intrusion Detection Datasets (2013) - for HIDS](https://www.unsw.adfa.edu.au/unsw-canberra-cyber/cybersecurity/ADFA-IDS-Datasets/)
- [ITOC CDX (2009)](https://westpoint.edu/centers-and-research/cyber-research-center/data-sets)
- [KDD-99 Dataset](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html)

## Dataset KDD-99

Meskipun dataset KDD99 sudah berusia lebih dari 20 tahun, dataset ini masih banyak digunakan untuk mendemonstrasikan Intrusion Detection Systems (IDS) dan Anomaly Detection. KDD99 merupakan dataset yang digunakan dalam The Third International Knowledge Discovery and Data Mining Tools Competition, yang diselenggarakan bersamaan dengan KDD-99, The Fifth International Conference on Knowledge Discovery and Data Mining.
Tugas dalam kompetisi tersebut adalah membangun sebuah pendeteksi intrusi jaringan, yaitu model prediktif yang mampu membedakan antara koneksi “buruk” (disebut intrusi atau serangan) dan koneksi “baik” (normal). Basis data ini berisi sekumpulan data standar untuk diaudit, termasuk berbagai jenis intrusi yang disimulasikan dalam lingkungan jaringan militer.

Kode berikut digunakan untuk membaca dataset KDD99 dalam format CSV ke dalam data frame Pandas. Format standar KDD99 tidak menyertakan nama kolom, sehingga program menambahkannya secara manual.

In [82]:
import pandas as pd
import urllib.request
import os

# Set Pandas display options
pd.set_option('display.max_columns', 6)
pd.set_option('display.max_rows', 5)

# Download the file using urllib
url = 'https://github.com/jeffheaton/jheaton-ds2/raw/main/kdd-with-columns.csv'
filename = 'kdd-with-columns.csv'

if not os.path.isfile(filename):
    try:
        urllib.request.urlretrieve(url, filename)
    except:
        print('Error downloading')
        raise

print(filename)

# Original file: http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html
df = pd.read_csv(filename)

print("Read {} rows.".format(len(df)))
# df = df.sample(frac=0.1, replace=False) # Uncomment this line to sample only 10% of the dataset
df.dropna(inplace=True, axis=1) 
# For now, just drop NA's (rows with missing values)

# Display 5 rows
pd.set_option('display.max_columns', 5)
pd.set_option('display.max_rows', 5)
print(df)

kdd-with-columns.csv
Read 494021 rows.
        duration protocol_type  ... dst_host_srv_rerror_rate  outcome
0              0           tcp  ...                      0.0  normal.
1              0           tcp  ...                      0.0  normal.
...          ...           ...  ...                      ...      ...
494019         0           tcp  ...                      0.0  normal.
494020         0           tcp  ...                      0.0  normal.

[494021 rows x 42 columns]


Dataset KDD99 berisi banyak kolom yang menggambarkan kondisi jaringan selama interval waktu tertentu, di mana serangan siber mungkin terjadi. Kolom “outcome” menunjukkan apakah koneksi tersebut “normal” (tidak ada serangan) atau jenis serangan yang dilakukan. Kode berikut menampilkan jumlah kemunculan untuk setiap jenis serangan dan data “normal”.

In [83]:
df.groupby('outcome')['outcome'].count()

outcome
back.               2203
buffer_overflow.      30
                    ... 
warezclient.        1020
warezmaster.          20
Name: outcome, Length: 23, dtype: int64

## Pemrosesan Data

Kita perlu melakukan beberapa tahap prapemrosesan sebelum data KDD99 dapat dimasukkan ke dalam jaringan saraf (neural network). Dua fungsi berikut disediakan untuk membantu proses prapemrosesan tersebut. Fungsi pertama mengonversi kolom numerik menjadi Z-Score. Fungsi kedua mengganti nilai kategorikal dengan variabel dummy.

In [84]:
import pandas as pd

def encode_numeric_zscore(df, name):
    """
    Apply z-score normalization to a specified numeric column.

    Parameters:
    df (DataFrame): The pandas DataFrame containing the column.
    name (str): The name of the column to normalize.
    """
    mean = df[name].mean()
    sd = df[name].std()
    df[name] = (df[name] - mean) / sd

def encode_text_dummy(df, name):
    """
    Convert a categorical column to dummy variables.

    Parameters:
    df (DataFrame): The pandas DataFrame containing the column.
    name (str): The name of the categorical column.
    """
    dummies = pd.get_dummies(df[name], prefix=name, dtype=float)
    df = pd.concat([df, dummies], axis=1)
    df.drop(name, axis=1, inplace=True)
    return df

def process_dataframe(df):
    """
    Process a DataFrame by encoding its features.

    Parameters:
    df (DataFrame): The pandas DataFrame to process.
    """
    for name in df.columns:
        if name == 'outcome':
            continue
        #elif df[name].dtype == bool:
        #    print("**", name)
        #    df[name] = df[name].astype(float)
        elif name in ['protocol_type', 'service', 'flag', 'land', 'logged_in',
                      'is_host_login', 'is_guest_login']:
            df = encode_text_dummy(df, name)
        else:
            encode_numeric_zscore(df, name)
    return df

Kode ini mengonversi semua kolom numerik menjadi Z-Score dan semua kolom teks menjadi variabel dummy. Selanjutnya, kita menggunakan fungsi-fungsi tersebut untuk melakukan prapemrosesan pada setiap kolom. Setelah program selesai memproses data, hasilnya kemudian ditampilkan.

In [85]:
pd.set_option('display.max_columns', 6)
pd.set_option('display.max_rows', 5)

df = process_dataframe(df)
df.dropna(inplace=True, axis=1)
print(df.head())

   duration  src_bytes  dst_bytes  ...  is_host_login_0  is_guest_login_0  \
0 -0.067792  -0.002879   0.138664  ...              1.0               1.0   
1 -0.067792  -0.002820  -0.011578  ...              1.0               1.0   
2 -0.067792  -0.002824   0.014179  ...              1.0               1.0   
3 -0.067792  -0.002840   0.014179  ...              1.0               1.0   
4 -0.067792  -0.002842   0.035214  ...              1.0               1.0   

   is_guest_login_1  
0               0.0  
1               0.0  
2               0.0  
3               0.0  
4               0.0  

[5 rows x 121 columns]


Kita membagi data menjadi dua kelompok, yaitu “normal” dan berbagai jenis serangan, untuk melakukan anomaly detection. Kode berikut membagi data tersebut ke dalam dua data frame dan menampilkan ukuran (jumlah data) dari masing-masing kelompok.

In [86]:
normal_mask = df['outcome']=='normal.'
attack_mask = df['outcome']!='normal.'

df.drop('outcome',axis=1,inplace=True)

df_normal = df[normal_mask]
df_attack = df[attack_mask]

print(f"Normal count: {len(df_normal)}")
print(f"Attack count: {len(df_attack)}")

Normal count: 97278
Attack count: 396743


Selanjutnya, kita mengonversi kedua data frame tersebut menjadi array Numpy.

In [87]:
# Ini adalah vektor fitur numerik, karena akan dimasukkan ke dalam jaringan saraf (neural net)
x_normal = df_normal.values
x_attack = df_attack.values

## Training Autoencoder

Penting untuk dicatat bahwa kita tidak menggunakan kolom outcome sebagai label yang akan diprediksi. Kita akan melatih autoencoder menggunakan data “normal” dan kemudian melihat seberapa baik model tersebut dapat mendeteksi bahwa data yang tidak diberi label “normal” merupakan anomali. Deteksi anomali ini bersifat unsupervised, artinya tidak ada nilai target (y) yang diprediksi.

Selanjutnya, kita membagi data normal menjadi 25% sebagai test set dan 75% sebagai train set. Program akan menggunakan data uji (test data) untuk memfasilitasi early stopping.

In [88]:
from sklearn.model_selection import train_test_split

x_normal_train, x_normal_test = train_test_split(x_normal, test_size=0.25, random_state=42)

Selanjutnya menampilkan ukuran (size) dari train set dan test set.

print(f"Normal train count: {len(x_normal_train)}")
print(f"Normal test count: {len(x_normal_test)}")

Sekarang kita siap untuk melatih autoencoder menggunakan data normal. Autoencoder akan belajar untuk memampatkan (compress) data menjadi sebuah vektor yang hanya terdiri dari tiga angka. Autoencoder juga seharusnya mampu melakukan dekompresi (decompress) kembali dengan tingkat akurasi yang wajar. Seperti halnya pada autoencoder pada umumnya, kita melatih jaringan saraf agar menghasilkan nilai keluaran (output) yang sama dengan nilai yang dimasukkan ke lapisan input.

In [89]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Convert numpy arrays to PyTorch tensors and move them to the appropriate device
x_normal_train_tensor = torch.tensor(x_normal_train).float().to(device)
x_normal_tensor = torch.tensor(x_normal).float().to(device)
x_attack_tensor = torch.tensor(x_attack).float().to(device)

# Create DataLoader for batch processing
train_data = TensorDataset(x_normal_train_tensor, x_normal_train_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

# Define the model using Sequential
model = nn.Sequential(
    nn.Linear(x_normal.shape[1], 25),
    nn.ReLU(),
    nn.Linear(25, 3),
    nn.ReLU(),
    nn.Linear(3, 25),
    nn.ReLU(),
    nn.Linear(25, x_normal.shape[1])
).to(device)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
# Training loop
for epoch in range(num_epochs):
    running_loss = 0
    den = 0
    for data in train_loader:
        inputs, targets = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss +=loss.item()
        den+=1

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss/den}')
    running_loss = 0.0

Using device: cpu
Epoch [1/10], Loss: 0.32353480313577804
Epoch [2/10], Loss: 0.2740984486387389
Epoch [3/10], Loss: 0.24666339941649584
Epoch [4/10], Loss: 0.2100492564005483
Epoch [5/10], Loss: 0.20960935956283816
Epoch [6/10], Loss: 0.19010148517207423
Epoch [7/10], Loss: 0.20308898677620546
Epoch [8/10], Loss: 0.18335483081397813
Epoch [9/10], Loss: 0.18626280895499675
Epoch [10/10], Loss: 0.16594917279782526


# Mendeteksi Anomali

Sekarang kita siap untuk melihat apakah data abnormal benar-benar merupakan anomali. Dua skor pertama menunjukkan kesalahan RMSE in-sample dan out-of-sample. Kedua skor ini relatif rendah, sekitar 0,33, karena berasal dari data normal. Sebaliknya, kesalahan yang jauh lebih tinggi, yaitu sekitar 0,76, muncul dari data abnormal. Autoencoder tidak mampu mengodekan data yang merepresentasikan serangan dengan baik. Kesalahan yang lebih tinggi ini menunjukkan adanya anomali.

In [90]:
model.eval()  # Set the model to evaluation mode

# Function to calculate RMSE
def calculate_rmse(model, data):
    with torch.no_grad():
        predictions = model(data)
        mse_loss = nn.MSELoss()(predictions, data)
    return torch.sqrt(mse_loss).item()

# Evaluating the model
score1 = calculate_rmse(model, torch.tensor(x_normal_test).float().to(device))
score2 = calculate_rmse(model, x_normal_tensor)
score3 = calculate_rmse(model, x_attack_tensor)

print(f"Skor Normal Out of Sample (di luar sampel) (RMSE): {score1}")
print(f"Skor Normal In-sample (dalam sampel) (RMSE): {score2}")
print(f"Skor Saat Serangan Berlangsung (RMSE): {score3}")

Skor Normal Out of Sample (di luar sampel) (RMSE): 0.3557773530483246
Skor Normal In-sample (dalam sampel) (RMSE): 0.37856152653694153
Skor Saat Serangan Berlangsung (RMSE): 0.5233561396598816


# Kesimpulan

Untuntuk menguji model, kami menggunakan pertama data normal x_normal_test, x_normal_tensor, dan x_attack_tensor. Kami menghitung skor RMSE untuk ketiga data ini dan membandingkannya dengan skor RMSE yang didapat sebelumnya. Hasilnya adalah:

Test score RMSE untuk data normal x_normal_test pertama

In [91]:
test_score_rmse_normal = calculate_rmse(model,torch.tensor(x_normal_test[0]).float().to(device))
print(f"Skor data pertama Normal Out of Sample (di luar sampel) (RMSE): {test_score_rmse_normal}")

Skor data pertama Normal Out of Sample (di luar sampel) (RMSE): 0.0649721696972847


Test score RMSE untuk data normal sampel x_normal_tensor pertama

In [92]:
test_score_rmse_sample_normal = calculate_rmse(model, x_normal_tensor[0])
print(f"Skor data pertama Normal In-sample (dalam sampel) (RMSE): {test_score_rmse_sample_normal}")

Skor data pertama Normal In-sample (dalam sampel) (RMSE): 0.1911206990480423


Test score RMSE untuk data pertama dari x_attack_tensor saat serangan terjadi.

In [93]:
test_score_rmse_attack = calculate_rmse(model,x_attack_tensor[0])
print(f"Skor data pertama saat Serangan Terjadi (RMSE): {test_score_rmse_attack}")

Skor data pertama saat Serangan Terjadi (RMSE): 8.7673978805542


Dari ketiga Score di atas terlihat bahwa Score RMSE untuk Data Normal akan menghasilkan Score yang rendah dibandingkan dengan Score RMSE untuk Data yang terkena Serangan. Hal ini karena Model yang dibangun sudah terlatih dengan Data Normal dan Data yang terkena Serangan. Oleh karena itu, Model yang dibangun sudah dapat mengenali Data yang terkena Serangan dan Data Normal. Hal ini dapat dilihat dari hasil Score RMSE yang cukup rendah untuk Data Normal dan cukup tinggi untuk Data yang terkena Serangan.

### Tentang Penulis

Kajian ini ditulis oleh :
- Nama : Doni Fristiyanto
- Nim : 241012000122 
- Kelas : v.340 (03MKME002)
- Jurusan : Magister Teknik Informatika
- Universitas : Universitas Pamulang
- Tanggal :  1 November 2025

